In [7]:
import argparse
import random
import cmath
import numpy as np
import matplotlib.pyplot as plt
import meep as mp

EPSILON_0 = 8.85418782e-12 # (m^-3 kg^-1 s^4 A^2) permitivity of free space
E = 1.60217662e-19 # (C) elementary charge
ME = 9.10938356e-31 # (kg) resting mass of electron
C = 299792458 # (m s^-1) speed of light

RESOLUTION = 5

In order to simulate the propagation of waves through the ionosphere, we must first know the permittivity and permeability of the space. In the plasma, we have a frequency-dependent permittivity $\epsilon(\omega)$. It is given as follows.

$$\omega_p = \sqrt{\frac{n_e e^2}{\epsilon_0 m_e}}$$

$$\epsilon(\omega) = 1 - \frac{\omega_p^2}{\omega^2}$$

In [8]:
def nde2pfreq(nde):
    '''Calculate the plasma frequency given the number density of electrons'''
    return np.sqrt(nde * E^2 / EPSILON_0 / ME)


def freqnde2permit(omega, nde):
    '''Calculate the permittivity given the transmitted frequency and the number density of electrons'''
    omegap = nde2freq(nde)
    return 1 - omegap ** 2 / omega ** 2

def freqnde2vel(omega, nde):
    epsilon = freqnde2permit(omega, nde)
    return C / sqrt(epsilon)

In [9]:
print('Starting')

# Characteristics of the Gaussian pulse
lambda_min = 0.4            # minimum source wavelength
lambda_max = 0.8            # maximum source wavelength
fmin = 1 / lambda_max         # minimum source frequency
fmax = 1 / lambda_min         # maximum source frequency
fcen = 0.5 * (fmin + fmax)      # source frequency center
df = fmax - fmin              # source frequency width

# Thicknesses of materials (lengths)
tABS = lambda_max           # absorber/PML thickness
tGLS = 1.0                  # glass thickness
tITO = 0.1                  # indium tin oxide thickness
tORG = 0.1                  # organic thickness
tAl = 0.1                   # aluminum thickness

# length of computational cell along Z
sz = tABS+tGLS+tITO+tORG+tAl

# length of non-absorbing region of computational cell in X and Y
L = 4.0 # args.L # TODO replace with a number
sxy = L + 2 * tABS
cell_size = mp.Vector3(sxy,sxy,sz)

# Boundary of the simulation (NOT a PML boundary!)
boundary_layers = [mp.Absorber(tABS, direction=mp.X),
                   mp.Absorber(tABS, direction=mp.Y),
                   mp.PML(tABS, direction=mp.Z, side=mp.High)]

# Define mediums by their index of refraction
ORG = mp.Medium(index=1.75)
ITO = mp.Medium(index=1.80)
GLS = mp.Medium(index=1.45)

from meep.materials import Al

geometry = [mp.Block(material=GLS, size=mp.Vector3(mp.inf,mp.inf,tABS+tGLS), center=mp.Vector3(0,0,0.5*sz-0.5*(tABS+tGLS))),
            mp.Block(material=ITO, size=mp.Vector3(mp.inf,mp.inf,tITO), center=mp.Vector3(0,0,0.5*sz-tABS-tGLS-0.5*tITO)),
            mp.Block(material=ORG, size=mp.Vector3(mp.inf,mp.inf,tORG), center=mp.Vector3(0,0,0.5*sz-tABS-tGLS-tITO-0.5*tORG)),
            mp.Block(material=Al, size=mp.Vector3(mp.inf,mp.inf,tAl), center=mp.Vector3(0,0,0.5*sz-tABS-tGLS-tITO-tORG-0.5*tAl))]

# current-source component (the "antenna")
# TODO take out all the args shit
if True: # args.perp_dipole:
    src_cmpt = mp.Ez
    symmetries = [mp.Mirror(mp.X,+1), mp.Mirror(mp.Y,+1)]
else:        
    src_cmpt = mp.Ex
    symmetries = [mp.Mirror(mp.X,-1), mp.Mirror(mp.Y,+1)]    
    
num_src = 10                 # number of point sources
sources = [];
for n in range(1, num_src):
    sources.append(mp.Source(mp.GaussianSource(fcen, fwidth=df), component=src_cmpt,
                             center=mp.Vector3(0,0,0.5*sz-tABS-tGLS-tITO-0.4*tORG-0.2*tORG*n/num_src),
                             amplitude=cmath.exp(2*cmath.pi*random.random()*1j)))

if True: # args.load_structure:
    epsilon_filename = 'oled_epsilon.h5'
    geometry = []
else:
    epsilon_filename = ''
        
sim = mp.Simulation(resolution=RESOLUTION,
                    cell_size=cell_size,
                    boundary_layers=boundary_layers,
                    geometry=geometry,
                    dimensions=3,
                    sources=sources,
                    force_complex_fields=True,
                    symmetries=symmetries)

# number of frequency bins for DFT fields
nfreq = 50
    
# surround source with a six-sided box of flux planes
srcbox_width = 0.05
srcbox_top = sim.add_flux(fcen, df, nfreq, 
                          mp.FluxRegion(center=mp.Vector3(0,0,0.5*sz-tABS-tGLS),
                                        size=mp.Vector3(srcbox_width,srcbox_width,0), direction=mp.Z, weight=+1))
srcbox_bot = sim.add_flux(fcen, df, nfreq,
                          mp.FluxRegion(center=mp.Vector3(0,0,0.5*sz-tABS-tGLS-tITO-0.8*tORG),
                                        size=mp.Vector3(srcbox_width,srcbox_width,0), direction=mp.Z, weight=-1))
srcbox_xp = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(center=mp.Vector3(0.5*srcbox_width,0,0.5*sz-tABS-tGLS-0.5*(tITO+0.8*tORG)),
                                       size=mp.Vector3(0,srcbox_width,tITO+0.8*tORG), direction=mp.X, weight=+1))
srcbox_xm = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(center=mp.Vector3(-0.5*srcbox_width,0,0.5*sz-tABS-tGLS-0.5*(tITO+0.8*tORG)),
                                       size=mp.Vector3(0,srcbox_width,tITO+0.8*tORG), direction=mp.X, weight=-1))
srcbox_yp = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(center=mp.Vector3(0,0.5*srcbox_width,0.5*sz-tABS-tGLS-0.5*(tITO+0.8*tORG)),
                                       size=mp.Vector3(srcbox_width,0,tITO+0.8*tORG), direction=mp.Y, weight=+1))
srcbox_ym = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(center=mp.Vector3(0,-0.5*srcbox_width,0.5*sz-tABS-tGLS-0.5*(tITO+0.8*tORG)),
                                       size=mp.Vector3(srcbox_width,0,tITO+0.8*tORG), direction=mp.Y, weight=-1))

# padding for flux box to fully capture waveguide mode
fluxbox_dpad = 0.05

# upward flux into glass substrate
glass_flux = sim.add_flux(fcen, df, nfreq,
                          mp.FluxRegion(center=mp.Vector3(0,0,0.5*sz-tABS-(tGLS-fluxbox_dpad)),
                                        size = mp.Vector3(L,L,0), direction=mp.Z, weight=+1))

# surround ORG/ITO waveguide with four-sided box of flux planes
# NOTE: waveguide mode extends partially into Al cathode and glass substrate
wvgbox_xp = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(size=mp.Vector3(0,L,fluxbox_dpad+tITO+tORG+fluxbox_dpad), direction=mp.X,
                                       center=mp.Vector3(0.5*L,0,0.5*sz-tABS-tGLS-0.5*(tITO+tORG)), weight=+1))
wvgbox_xm = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(size=mp.Vector3(0,L,fluxbox_dpad+tITO+tORG+fluxbox_dpad), direction=mp.X,
                                       center=mp.Vector3(-0.5*L,0,0.5*sz-tABS-tGLS-0.5*(tITO+tORG)), weight=-1))
wvgbox_yp = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(size=mp.Vector3(L,0,fluxbox_dpad+tITO+tORG+fluxbox_dpad), direction=mp.Y,
                                       center=mp.Vector3(0,0.5*L,0.5*sz-tABS-tGLS-0.5*(tITO+tORG)), weight=+1))
wvgbox_ym = sim.add_flux(fcen, df, nfreq,
                         mp.FluxRegion(size=mp.Vector3(L,0,fluxbox_dpad+tITO+tORG+fluxbox_dpad), direction=mp.Y,
                                       center=mp.Vector3(0,-0.5*L,0.5*sz-tABS-tGLS-0.5*(tITO+tORG)), weight=-1))

print('Done')

Starting
Done


In [12]:
print('Starting')

#sim.run(until_after_sources=mp.stop_when_fields_decayed(50, src_cmpt, mp.Vector3(0,0,0.5*sz-tABS-tGLS-tITO-0.5*tORG), 1e-8))
sim.run(until=5000)
eps_data = sim.get_array(center=mp.Vector3(), size=cell_size, component=mp.Dielectric)

from mayavi import mlab
s = mlab.contour3d(eps_data, colormap="YlGnBu")
mlab.show()

sim.dump_structure('oled_epsilon.h5')

print('Done')

Starting
Meep progress: 92.19980468750055/12620.7001953125 = 0.7% done in 4.0s, 543.7s to go
Meep progress: 191.09980468750018/12620.7001953125 = 1.5% done in 8.0s, 520.5s to go
Meep progress: 289.7998046875/12620.7001953125 = 2.3% done in 12.0s, 510.8s to go
Meep progress: 371.0998046875002/12620.7001953125 = 2.9% done in 16.0s, 528.4s to go
Meep progress: 440.4998046875007/12620.7001953125 = 3.5% done in 20.0s, 553.3s to go
Meep progress: 523.0998046875002/12620.7001953125 = 4.1% done in 24.0s, 555.3s to go
Meep progress: 606.4998046875007/12620.7001953125 = 4.8% done in 28.0s, 554.9s to go
Meep progress: 683.7998046875/12620.7001953125 = 5.4% done in 32.0s, 558.9s to go
Meep progress: 763.7998046875/12620.7001953125 = 6.1% done in 36.0s, 559.1s to go
Meep progress: 837.1998046874996/12620.7001953125 = 6.6% done in 40.0s, 563.2s to go
Meep progress: 917.7998046875/12620.7001953125 = 7.3% done in 44.0s, 561.3s to go
Meep progress: 990.0998046875011/12620.7001953125 = 7.8% done in 48.0

ModuleNotFoundError: No module named 'vtkIOParallelPython'

In [ ]:
print('Start')

flux_srcbox_top = np.asarray(mp.get_fluxes(srcbox_top))
flux_srcbox_bot = np.asarray(mp.get_fluxes(srcbox_bot))
flux_srcbox_xp = np.asarray(mp.get_fluxes(srcbox_xp))
flux_srcbox_xm = np.asarray(mp.get_fluxes(srcbox_xm))
flux_srcbox_yp = np.asarray(mp.get_fluxes(srcbox_yp))
flux_srcbox_ym = np.asarray(mp.get_fluxes(srcbox_ym))

flux_wvgbox_xp = np.asarray(mp.get_fluxes(wvgbox_xp))
flux_wvgbox_xm = np.asarray(mp.get_fluxes(wvgbox_xm))
flux_wvgbox_yp = np.asarray(mp.get_fluxes(wvgbox_yp))
flux_wvgbox_ym = np.asarray(mp.get_fluxes(wvgbox_ym))

flux_glass = np.asarray(mp.get_fluxes(glass_flux))

flux_total = flux_srcbox_top+flux_srcbox_bot+flux_srcbox_xp+flux_srcbox_xm+flux_srcbox_yp+flux_srcbox_ym
flux_waveguide = flux_wvgbox_xp+flux_wvgbox_xm+flux_wvgbox_yp+flux_wvgbox_ym

frac_glass = flux_glass/flux_total
frac_waveguide = flux_waveguide/flux_total
frac_aluminum = 1-frac_glass-frac_waveguide

freqs = np.asarray(mp.get_flux_freqs(glass_flux))
lambdas = 1/freqs
lambdas_linear = np.linspace(lambda_min,lambda_max,nfreq)

from scipy import interpolate

g_linear = interpolate.interp1d(lambdas,frac_glass,kind='cubic')
w_linear = interpolate.interp1d(lambdas,frac_waveguide,kind='cubic')
a_linear = interpolate.interp1d(lambdas,frac_aluminum,kind='cubic')
frac_glass_linear = g_linear(lambdas_linear)
frac_waveguide_linear = w_linear(lambdas_linear)
frac_aluminum_linear = a_linear(lambdas_linear)

for j in range(nfreq):
    print("data:, {:.4f}, {:.6f}, {:.6f}, {:.6f}".format(lambdas_linear[j],
                                                         frac_glass_linear[j],
                                                         frac_waveguide_linear[j],
                                                         frac_aluminum_linear[j]))

print('Done')

In [5]:
# import matplotlib.pyplot as plt

# f = np.genfromtxt("oled-flux.dat", delimiter=",")
# lambdas = f[:,0]
# glass = f[:,1]
# waveguide = f[:,2]
# aluminum = f[:,3]

# plt.plot(lambdas,glass,'b-',label='glass')
# plt.plot(lambdas,waveguide,'r-',label='organic + ITO')
# plt.plot(lambdas,aluminum,'g-',label='aluminum')
# plt.xlabel("wavelength (um)"); plt.ylabel("fraction of total power")
# plt.axis([0.4, 0.8, 0, 1])
# plt.xticks([t for t in arange(0.4,0.9,0.1)])
# plt.legend(loc='upper right')
# plt.show()

# print("glass: {} (mean), {} (std. dev.)".format(np.mean(glass),np.std(glass)))
# print("waveguide: {} (mean), {} (std. dev.)".format(np.mean(waveguide),np.std(waveguide)))
# print("aluminum: {} (mean), {} (std. dev.)".format(np.mean(aluminum),np.std(aluminum)))